# Drug Prescription Effectiveness and Ratings
### Dataset 
This dataset deals with prescription drugs; it was found on Kaggle from user Rohan Harode.

Link: https://www.kaggle.com/datasets/jessicali9530/kuc-hackathon-winter-2018

#### Attributes
* _Drug_: Name of drug
* _condition_: Name of condition the drug intends to treat
* _date_: date of review and rating entry
* _Effectiveness_: 5 star patient review
* _Age_: age range of the patient
* _EaseOfUse_: 5 star rating of how easy the drug is to use
* _Satisfaction_: 5 star rating of how the patients liked the drug
* _Sex_: gender of patient

* _*Both 'usefulCount' and 'review' attributes were ommitted; they will serve no use in the predictions_

#### Predictions
We are trying to predict the effectiveness of each drug based on an unseen patient. To do this, we will need to find the average effectiveness of each drug grouped by drug name.

In [71]:
# some useful mysklearn package import statements and reloads
import importlib
import mysklearn.myutils
importlib.reload(mysklearn.myutils)
import mysklearn.myutils as myutils

# uncomment once you paste your mypytable.py into mysklearn package
import mysklearn.mypytable
importlib.reload(mysklearn.mypytable)
from mysklearn.mypytable import MyPyTable 

# uncomment once you paste your myclassifiers.py into mysklearn package
import mysklearn.myclassifiers
importlib.reload(mysklearn.myclassifiers)
from mysklearn.myclassifiers import MyKNeighborsClassifier, MyDummyClassifier, MyNaiveBayesClassifier, MyDecisionTreeClassifier

import mysklearn.myevaluation
importlib.reload(mysklearn.myevaluation)
import mysklearn.myevaluation as myevaluation

In [86]:
# Dataset Preprocessing
table, header = myutils.get_tables("input_data/webmd.csv")
drug_table = MyPyTable(header, table)

drug_table.remove_rows_with_missing_values()
drug_table.convert_to_numeric()
to_rem = drug_table.find_duplicates(['Drug','Age','Condition','Date','EaseofUse','Satisfaction','Sex','UsefulCount','Effectiveness'])
drug_table.drop_rows(to_rem)

# remove non-useful attributes
for i in range(len(drug_table.data)):
    del drug_table.data[i][7]

del drug_table.column_names[7]

# convert dates to seasons, change some misc age values
drug_table.column_names[3] = 'Season'
for i in range(len(drug_table.data)):
    val = drug_table.data[i][3]
    newVal = myutils.season_discretize(val)
    drug_table.data[i][3] = newVal

    if drug_table.data[i][1] == '6-Mar':
        drug_table.data[i][1] = '3-6'
    if drug_table.data[i][1] == '12-Jul':
        drug_table.data[i][1] = '7-12'

drug_table.save_to_file('input_data/clean_drug.csv')

In [108]:
# stats by - group by 'Drug'
import math

values, counts = drug_table.get_frequencies('Drug')

grouped_data = []
for item in values:
    grouped_data.append([])

for item in drug_table.data:
    ind = values.index(item[0])
    grouped_data[ind].append(item)

drug_avg_data = []
for i in range(len(grouped_data)):
    instance = []
    instance.append(grouped_data[i][0][0])

    '''avg_age = []
    dom_cond = []
    dom_season = []
    avg_ease = []
    avg_satis = []
    dom_gender = []
    avg_eff = []'''

    mean_lists = []
    for i in range(7):
        mean_lists.append([])

    print(mean_lists)

    for item in grouped_data[i]:
        for j in range(len(item)):
            if j == 0:
                pass
            else:
                mean_lists[j].append(item[j])
    
    print(mean_lists)

    drug_avg_data.append(instance)

IndexError: list index out of range

In [7]:


# get X and y for stratified k fold cross validation
X = []
y = []
for i in range(len(drug_table.data)):
    # create X, y, remove class label from X
    X.append(drug_table.data[i]) 
    y.append(drug_table.data[i][-1]) # class label
# remove class label from X
for row in X:
    row.pop(-1)

# get folds
X_train_folds, X_test_folds = myevaluation.stratified_kfold_cross_validation(X, y, n_splits=5, random_state=None, shuffle=True)

# create training/testing sets out of the folds
X_train_sets, y_train_sets = myutils.get_sets_from_folds(X, y, X_train_folds)
X_test_sets, y_test_sets = myutils.get_sets_from_folds(X, y, X_test_folds)

labels = [5, 4, 3, 2, 1] # has to be numeric bc the vals in the table are too


# k_neighbors = MyKNeighborsClassifier(n_neighbors=7)
# myutils.fit_predict_classification(X_train_sets, y_train_sets, 
#     X_test_sets, y_test_sets, k_neighbors, "K Neighbors", labels)

dummy = MyDummyClassifier()
myutils.fit_predict_classification(X_train_sets, y_train_sets, 
    X_test_sets, y_test_sets, dummy, "Dummy", labels)

# tree = MyDecisionTreeClassifier()
# myutils.fit_predict_classification(X_train_sets, y_train_sets, 
#     X_test_sets, y_test_sets, tree, "Tree", labels)

Dummy Classification
Accuracy: 0.46 ~ Error Rate: 0.54
Recall Score: 1.0
Precision Score: 0.43
F1 Score: 0.6

Confusion Matrix:
+-----+---+---+---+---+
|  5  | 4 | 3 | 2 | 1 |
+-----+---+---+---+---+
| 286 | 0 | 0 | 0 | 0 |
| 170 | 0 | 0 | 0 | 0 |
| 90  | 0 | 0 | 0 | 0 |
| 48  | 0 | 0 | 0 | 0 |
| 64  | 0 | 0 | 0 | 0 |
+-----+---+---+---+---+
